<a href="https://colab.research.google.com/github/qrak/3commas-TakeProfit-Update/blob/master/lstmlastchance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorboard

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from pytorch_lightning.loggers import TensorBoardLogger
import matplotlib.pyplot as plt

def create_input_target_tensors(df, sequence_length, target_column):
    input_data = []
    target_data = []

    for i in range(len(df) - sequence_length):
        input_data.append(df.iloc[i:i+sequence_length].drop(columns=[target_column]).values)
        target_data.append(df.iloc[i+sequence_length, df.columns.get_loc(target_column)])

    input_tensor = torch.tensor(input_data, dtype=torch.float32)
    # Add an additional dimension to the target tensor
    target_tensor = torch.tensor(target_data, dtype=torch.float32).unsqueeze(1)

    return input_tensor, target_tensor




class SelfAttention(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SelfAttention, self).__init__()
        self.W_query = nn.Linear(input_dim, output_dim)
        self.W_key = nn.Linear(input_dim, output_dim)
        self.W_value = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        query = self.W_query(x)
        key = self.W_key(x)
        value = self.W_value(x)

        attention_weights = F.softmax(torch.matmul(query, key.transpose(-2, -1)) / (query.size(-1) ** 0.5), dim=-1)
        attention_output = torch.matmul(attention_weights, value)
        return attention_output

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.attention_heads = nn.ModuleList([SelfAttention(input_dim, output_dim) for _ in range(num_heads)])

    def forward(self, x):
        attention_outputs = [head(x) for head in self.attention_heads]
        return torch.cat(attention_outputs, dim=-1)

class BitcoinPredictor(pl.LightningModule):
    def __init__(self, input_size, hidden_size, output_size, num_layers, sequence_length, num_heads):
        super(BitcoinPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=0.5)
        self.multi_head_attention = MultiHeadSelfAttention(2 * hidden_size, hidden_size, num_heads)
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(hidden_size * num_heads, output_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.multi_head_attention(x)
        x = self.dropout(x)
        x = self.linear(x[:, -1])
        return x


    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = F.mse_loss(y_pred, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = F.mse_loss(y_pred, y)
        self.log('val_loss', loss, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer
        
def normalize_data(df):
    timestamp_column = 'timestamp'  # Change this to match the name of your timestamp column
    scaler = MinMaxScaler()

    # Remove the timestamp column
    df_no_timestamp = df.drop(columns=[timestamp_column])

    # Normalize the remaining columns
    df_normalized = pd.DataFrame(scaler.fit_transform(df_no_timestamp), columns=df_no_timestamp.columns)

    return df_normalized


def train_val_split(df, train_ratio=0.8):
    split_idx = int(len(df) * train_ratio)
    return df[:split_idx], df[split_idx:]

def plot_predictions(actual, predicted):
    plt.figure(figsize=(14, 8))
    plt.plot(actual, label='Actual', color='blue')
    plt.plot(predicted, label='Predicted', color='orange')
    plt.xlabel('Time steps')
    plt.ylabel('Bitcoin Price')
    plt.legend()
    plt.show()

def evaluate_and_plot(model, val_loader):
    model.eval()
    all_actual_values = []
    all_predicted_values = []

    with torch.no_grad():
        for batch in val_loader:
            x, y = batch
            y_pred = model(x)

            all_actual_values.extend(y.numpy())
            all_predicted_values.extend(y_pred.numpy())

    plot_predictions(all_actual_values, all_predicted_values)

def main():
    # Load and normalize your historical OHLCV data and indicators here
    df = pd.read_csv('BTC_USDT_1h_indicators.csv')
    df = normalize_data(df)

    sequence_length = 24
    train_df, val_df = train_val_split(df)

    batch_size = 64

    # Prepare the input and target tensors
    train_input, train_target = create_input_target_tensors(train_df, sequence_length, 'close')
    val_input, val_target = create_input_target_tensors(val_df, sequence_length, 'close')

    # Create the TensorDatasets
    train_dataset = TensorDataset(train_input, train_target)
    val_dataset = TensorDataset(val_input, val_target)

    # Create the DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

    input_size = train_df.shape[1] - 1
    hidden_size = 128
    output_size = 1
    num_layers = 2
    num_heads = 4

    model = BitcoinPredictor(input_size, hidden_size, output_size, num_layers, sequence_length, num_heads)
    logger = TensorBoardLogger('tb_logs', name='bitcoin_predictor')
    checkpoint_callback = ModelCheckpoint(monitor='val_loss', mode='min', save_top_k=1)

    trainer = pl.Trainer(
        max_epochs=10,
        accelerator='cuda' if torch.cuda.is_available() else 0,
        log_every_n_steps=1,
        logger=logger,
        callbacks=[checkpoint_callback]
        )
    trainer.fit(model, train_loader, val_loader)
    # Evaluate and plot predictions
    evaluate_and_plot(model, val_loader)    

if __name__ == '__main__':
    main()



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                 | Type                   | Params
----------------------------------------------------------------
0 | lstm                 | LSTM                   | 547 K 
1 | multi_head_attention | MultiHeadSelfAttention | 394 K 
2 | dropout              | Dropout                | 0     
3 | linear               | Linear                 | 513   
----------------------------------------------------------------
943 K     Trainable params
0         Non-trainable params
943 K     Total params
3.772     Total estimated model params 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]